# Lektion 10 - Forenklade processer (Lightning / fastai)

**Assignment: Reduce boilerplate in training loops**

Instructions:
1. Start from a plain PyTorch training loop
2. Refactor to Lightning OR implement with fastai
3. Compare code length and readability

## Task 1: Baseline PyTorch loop
Start with a plain training loop.

In [23]:
# TODO: Build a small model and training loop in PyTorch
# Det här gör vi som vanligt! 
# Vi laddar in data, sen bygger en träningsloop

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Vi laddar in data och splittar i X och y
iris = load_iris()
X = iris.data
y = iris.target

# Vi delar in datan i train och test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

# Vi scalar vår data efter splitten, för att undvika data läckage
# Vi skulle kunna skapa en Scaler, och sedan återanvända, men då
# får vi se till att inte köra fit på bägge, så inget data smiter över
X_train = StandardScaler().fit_transform(X_train)
X_test = StandardScaler().fit_transform(X_test)

# ============ HÄR SKULLE VI TYPISKT GÖRA EDA ============ #

# vi definerar en DL-modell med 3 lager (in - hidden - out)
# De har (4, 128 respektive 3 noder)
model = nn.Sequential(
    nn.Linear(4, 128),
    nn.ReLU(),
    nn.Linear(128, 3)
)

# Vi använder crossentropyloss, eftersom vi har ett klassifikationsproblem med >2 klasser
criterion = nn.CrossEntropyLoss()

# adam är vår standardoptimerare!
optimizer = optim.Adam(model.parameters(), lr=0.01)

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

model.to(device)


Sequential(
  (0): Linear(in_features=4, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=3, bias=True)
)

In [24]:
X.shape

(150, 4)

In [25]:
# Vi gör om vår data till tensors, och packeterar i TensorDataset och DataLoader för att underlätta träning

train_ds = TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(y_train, dtype=torch.long),
)
test_ds = TensorDataset(
    torch.tensor(X_test, dtype=torch.float32),
    torch.tensor(y_test, dtype=torch.long),
)

# Loadern gör att vi kan iterera över vår data i batches, och även shuffla den under träning
# Detta är också något som underlättar för torch att hantera datan, och kan leda till bättre konvergens
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=16)

In [27]:
# TODO: Train for a few epochs and record accuracy

epochs = 3
for _ in range(epochs):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        
        # A. Zero grad: Set gradients to zero before backward pass
        optimizer.zero_grad()
    
        # B. Forward: Build the graph & get prediction
        # Outputs kan ofta kallas logits, men det är inget måste
        outputs = model(xb)
        loss = criterion(outputs, yb)
    
        # C. Backward: AutoDiff calculates the "blame" (gradients)
        loss.backward()
        
        # D. Update: Optimizer moves weights down the hill
        optimizer.step()

model.eval()
correct, total = 0, 0
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        preds = torch.argmax(model(xb), dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)
print(f"Baseline accuracy: {correct / total:.4f}")

Baseline accuracy: 0.8889


## Task 2: Refactor with Lightning OR fastai
Reduce boilerplate using a higher-level framework.

In [ ]:
# TODO: Convert the loop into a LightningModule (or a fastai Learner)

In [ ]:
# TODO: Train the same model and record accuracy

## Task 3: Compare
Reflect on readability and debugging.

In [ ]:
# TODO: Write 4-6 comment lines about:
# - what boilerplate disappeared
# - what became easier or harder to debug

In [ ]:
print("Done! You simplified training with higher-level tools.")